In [1]:
# First, convert the json into python dictionary
import json
    
## read the json file
f = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 1\\\\shop_dict.json')
data = json.load(f)

In [2]:
data['newegg.com']

{'product_detail': {'shop': 'newegg.com',
  'url': 'http://www.newegg.com/Product/Product.aspx?Item=9SIA1J70TT0274',
  'modelID': 'UN46ES7100FXZA',
  'featuresMap': {'Analog Audio': ['1 In / 1 Out',
    '3 In',
    '1 Out',
    '1 In',
    '3 In / 1 Out',
    'Analog Audio Input(s) (Total): 1 (Rear)',
    '1 (L + R)',
    '2 x L/R in',
    'Yes',
    '2 In / 1 Out'],
   'Component Video': ['1 In',
    'HD Component x 1',
    'Yes',
    '2 In',
    '1 (rear) 1 (side w/gender)',
    '1 (Rear)',
    '2',
    '1',
    'RCA Component (x2)',
    '2 HD Component (Y-Pb-Pr with R/L audio) Inputs ( 1 Combo Composite Capable)',
    '2 (Rear)',
    '3 In',
    '1 (rear)',
    '2 (1 Side, 1 Rear)',
    '1 In (shared with composite)',
    '1 x RCA-YPbPr',
    'RCA x 1',
    '1 (rear), 1 (side w/gender)',
    '1 (Rear/Hybrid)',
    '1 Side, 2 Rear',
    '1 In (rear) + 1 In (side w/gender)',
    '1 Component Video (Shared Audio)'],
   'LAN': ['Ethernet In',
    '1',
    'Ethernet x 1',
    '2',
    'E

In [3]:
# define the frequency function to calculate the data dimension
def most_frequent(List):
    return max(set(List), key = List.count)

In [4]:
common_unit = ['inches','"','lbs','lb','kg','cd/m','Nit','$','pounds','mm','ms','W','V']

In [5]:
import re

# figure out the key type is str or double
for shop in data.values():
    for key in shop['product_detail']['featuresMap'].keys():
        valuetype = {'string':0,'double':0}
        doubleValues = []
        dimension = []
        for i in range(len(shop['product_detail']['featuresMap'][key])):
            # select each value, extract its digit element
            value = shop['product_detail']['featuresMap'][key][i]
            numbers = [float(s) for s in re.findall(r'\d+\.\d+|\d+', value)]
            doubleValues.append(numbers)
            dimension.append(len(numbers))
            
            # define the lenth of value
            value_len = len(value)
            
            # define the length of digit element
            num_len = 0
            for i in range(len(numbers)):
                num_len += len(str(numbers[i]))
                
            # define the data type of each element
            if num_len > value_len*0.5:
                valuetype['double'] += 1 
            elif any(unit in value for unit in common_unit):
                valuetype['double'] += 1
            else: valuetype['string'] += 1 
        
        # now compare the count of valuetype
        if valuetype['double'] >= valuetype['string']:
            keytype = 'double'
        else :
            keytype = 'string'
            
        # prepare the unique value list
        rawValues = list(shop['product_detail']['featuresMap'][key])
        if keytype == 'string':
            stringValues = list(shop['product_detail']['featuresMap'][key])
            doubleValues = 'None'
            dimension = 'None'
        else:
            stringValues = 'None'
            dimension = most_frequent(dimension)
        
        diversity = len(shop['product_detail']['featuresMap'][key])
            
        # reform the key matrix    
        shop['product_detail']['featuresMap'][key] = dict({'keytype':keytype,'Raw Value':rawValues,'String Values':stringValues,'Double Values':doubleValues,'Dimension':dimension,'Diversity':diversity})
    

In [6]:
# remove the block double data 

for shop in data.values():
    for key in shop['product_detail']['featuresMap'].values():
        if key['keytype'] == 'double':
            key['Double Values'] = [x for x in key['Double Values'] if x != []]
        else: pass
                

In [7]:
data['bestbuy.com']

{'product_detail': {'shop': 'bestbuy.com',
  'url': 'http://www.bestbuy.com/site/NEC+-+42%22+Class+(42%22+Diag.)+-+LED-LCD+TV+-+1080p+-+HDTV+1080p/1308511267.p;template=_specificationsTab',
  'modelID': 'E424',
  'featuresMap': {'Meta Title': {'keytype': 'string',
    'Raw Value': ['NEC Display Solutions E424',
     'Philips Electronics 39PFL2908',
     'Samsung HG40NA577LF',
     'Samsung UN46F5000AF',
     'NEC Display Solutions E463',
     'Samsung H40B',
     'LG Electronics 47G2',
     'Panasonic TH-42PH30U',
     'ORION Electric Co., Ltd SLED3280',
     'Coby Electronics Corporation LEDTV4626',
     'Vizio, Inc E371VL',
     'Viewsonic Corporation VT2215LED',
     'Toshiba 65HT2U',
     'Philips Electronics 32PFL4508',
     'SunBriteTV, LLC 3260HD',
     'Samsung UN40F5000AF',
     'Toshiba 40L2200U',
     'Vizio, Inc E261VA',
     'LG Electronics E1910T-BN',
     'Vizio, Inc M470SL',
     'LG Electronics 50PM9700',
     'Samsung H32B',
     'Haier America LE50F2280',
     'NEC D

In [8]:
# get std for all double key
import numpy as np

for shop in data.values():
    for key in shop['product_detail']['featuresMap'].keys():
        # if not double value the pass
        if shop['product_detail']['featuresMap'][key]['keytype'] == 'string':
            pass
        # select all the double value key
        else:
            dimension = shop['product_detail']['featuresMap'][key]['Dimension']
            std = []
            
            # select all the product value only in the same dimension with the key
            for numbers in shop['product_detail']['featuresMap'][key]['Double Values']:
                if len(numbers) == dimension:
                    std.append(numbers)
                else: pass
            
            if std == []:
                std = ['None']
            if len(std[0]) > 1:
                std_in = []
                position = 0
                while position < dimension:
                    #creat the parameter
                    number_in = []
                    
                    for i in std:
                        number_in.append(i[position])
                    
                    std_in.append(np.std(number_in))
                    position += 1
                    
                Standard_Deviation = list(std_in)
            elif std[0] == 'None':
                Standard_Deviation = 'None'
                    
            else: 
                Standard_Deviation = np.std(std)
            
            shop['product_detail']['featuresMap'][key]['std'] = Standard_Deviation

In [9]:
output = {}
for shop in data.keys():
    output[shop] = data[shop]['product_detail']['featuresMap']

In [10]:
import json
with open('shop_dict_A2.json', 'w') as f:
    json.dump(output, f)